# VREF Filter

Objective: Design a Digital (PWM) to Analog filter with fast response time for setting voltage references in motor drives.




In [28]:
using Pkg
Pkg.add("ControlSystems")
Pkg.add("Plots")
Pkg.add("PlotlyBase")
Pkg.add("DifferentialEquations")

  Resolving package versions...
No Changes to `~/Ultimachine/JupyterNotebooks/VREF_Filter/Project.toml`
No Changes to `~/Ultimachine/JupyterNotebooks/VREF_Filter/Manifest.toml`
  Resolving package versions...
No Changes to `~/Ultimachine/JupyterNotebooks/VREF_Filter/Project.toml`
No Changes to `~/Ultimachine/JupyterNotebooks/VREF_Filter/Manifest.toml`
  Resolving package versions...
No Changes to `~/Ultimachine/JupyterNotebooks/VREF_Filter/Project.toml`
No Changes to `~/Ultimachine/JupyterNotebooks/VREF_Filter/Manifest.toml`
  Resolving package versions...
No Changes to `~/Ultimachine/JupyterNotebooks/VREF_Filter/Project.toml`
No Changes to `~/Ultimachine/JupyterNotebooks/VREF_Filter/Manifest.toml`


In [96]:
using ControlSystems
using Plots
using PlotlyBase
#using DifferentialEquations
plotly(); #nicer plots backend when running live
#gr(); # use GR for publish

Construct the transfer function of an RC low pass filter given by:

$$
\frac{1}{1+RCs}
$$

In [97]:
function lowpass_filter(R,C)
    s = tf("s")
    1/(1+R*C*s)
end

lowpass_filter (generic function with 1 method)

In [98]:
24_000_000/2^9

46875.0

In [99]:
"""
Generate a PWM signal given a base clock and resolution (in bits).

clock::Int e.g. 24_000_000
resolution::Int e.g. 8 bit
duty_cycles::Vector{Int} e.g. [0, 128, 255, 64]
"""
function PWM(clock, resolution, values)

    period = resolution/clock
    bitperiod = period/(2^resolution)
    delays = DelayLtiSystem{Float64,Float64}[]
    
    for i in eachindex(values)
        offset = (i-1)*period
        push!(delays, delay(offset)) # initial
        push!(delays, -delay(offset+values[i]*bitperiod)) # turnoff
    end

    return sum(delays)
end

PWM

In [100]:
# Step function for LTI sim
function u0(out,t)
    if t > 0
        out[1] = 1 #Volts, but doesnt really matter so it is easier to see duty cycle
    else
        out[1] = 0
    end
    return
end

u0 (generic function with 1 method)

In [120]:
# 1.0b filter params

R = 1e2
C = 4.7e-9

@show 1/(R*C)

f1 = lowpass_filter(R,C)
f2 = lowpass_filter(R,C)
f3 = f1*f2




1 / (R * C) = 2.127659574468085e6


TransferFunction{Continuous,ControlSystems.SisoRational{Float64}}
             1.0
------------------------------
2.209e-13*s^2 + 9.4e-7*s + 1.0

Continuous-time transfer function model

In [124]:
input = PWM(27_000_000, 9, cat(fill(128*2, 50) ,fill(64*2, 100),fill(255*2, 100), fill(25*2, 100), dims=1))

sys = ss(f3)*input

t = 0:5e-8:(input.Tau[end])


@time y, t, x = lsim(sys, u0, t, alg=ControlSystems.MethodOfSteps(BS3()))

 14.071207 seconds (94.05 M allocations: 3.052 GiB, 2.84% gc time)


([0.0; 0.004765054200150471; … ; 0.10104714365630771; 0.09882634146932368], 0.0:5.0e-8:0.00011635, [0.0 0.0; 1.052600472813239e-15 1.3460967758161054e-8; … ; 2.2321314033678375e-14 -1.051878966287259e-8; 2.18307388305736e-14 -4.4524030909125017e-10])

In [125]:
plot(t, y)

<!DOCTYPE html>
 
 
 Plots.jl

In [126]:
bodeplot(f3)

<!DOCTYPE html>
 
 
 Plots.jl

In [127]:
bodeplot(f1)

<!DOCTYPE html>
 
 
 Plots.jl